In [9]:
import pandas as pd
import pyodbc
from IPython.display import clear_output

# 1) Cargar archivo Excel
df = pd.read_excel(
    r'C:\Users\rrs23\OneDrive\Documentos\OneDrive\Documentos\Datos\BaseQ.xlsx',
    sheet_name='Base Principal'
)

# 2) Limpiar columnas
df.columns = df.columns.str.strip()
df.rename(columns={'Ciudad': 'ciudad'}, inplace=True)

df['codigo_cedi'] = df['codigo_cedi'].astype(str).str.strip()
df['direccion_servicio'] = df['direccion_servicio'].astype(str).str.strip()
df['ciudad'] = df['ciudad'].astype(str).str.strip().replace('', 'NULL')

# 3) Filtrar columnas necesarias y quitar duplicados
df_direcciones = df[['codigo_cedi', 'direccion_servicio', 'ciudad']].drop_duplicates()

# 4) Conectar a SQL Server
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost;'
    'DATABASE=quick;'
    'Trusted_Connection=yes;'
)
cursor = conn.cursor()

# 5) Insertar en DireccionesCedis
insertados = 0
for i, row in df_direcciones.iterrows():
    try:
        cursor.execute("""
            INSERT INTO DireccionesCedis (codigo_cedi, direccion_servicio, ciudad)
            VALUES (?, ?, ?)
        """, row.codigo_cedi, row.direccion_servicio, row.ciudad if row.ciudad != 'NULL' else None)
        insertados += 1

        if (i + 1) % 50 == 0:
            clear_output(wait=True)
            print(f"Direcciones procesadas: {i + 1}")

    except Exception as e:
        print(f"Error al insertar dirección para {row.codigo_cedi}: {e}")

conn.commit()
conn.close()

print(f"Direcciones insertadas: {insertados}")




Direcciones procesadas: 7050
Direcciones insertadas: 437
